<a href="https://colab.research.google.com/github/Nadjei74/Collab/blob/main/GA2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
from numpy.random import randint



In [ ]:
# Objective Function
def objective(x):
    return sum(x)



In [ ]:
# Decode continuous values
def decode_cont(bounds_cont, n_bits_cont, bitstring):
    decoded_cont = []
    largest = 2 ** n_bits_cont
    for i in range(len(bounds_cont)):
        start, end = i * n_bits_cont, (i + 1) * n_bits_cont
        substring = bitstring[start:end]
        chars = ''.join([str(s) for s in substring])
        integer = int(chars, 2)
        value = bounds_cont[i][0] + (integer / largest) * (bounds_cont[i][1] - bounds_cont[i][0])
        decoded_cont.append(value)
    return decoded_cont




In [ ]:
# Decode discrete values
def decode_discrete(bounds_discret, n_bits_discret, bitstring):
    decoded_discrete = []
    for i in range(bounds_discret):
        start, end = i * n_bits_discret, (i + 1) * n_bits_discret
        substring = bitstring[start:end]
        chars = ''.join([str(s) for s in substring])
        value = int(chars, 2)  # Assuming discrete values are integer representations
        decoded_discrete.append(value)
    return decoded_discrete



In [ ]:
# Tournament Selection
def selection(pop, scores, k=3):
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k - 1):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]



In [ ]:
# Crossover
def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if random.random() < r_cross:
        pt = randint(1, len(p1) - 2)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]



In [ ]:
# Mutation
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if random.random() < r_mut:
            bitstring[i] = 1 - bitstring[i]
    return bitstring



In [ ]:
# Genetic Algorithm
def genetic_algorithm(objective, bounds_cont, n_bits_cont, bounds_discret, n_bits_discret, n_iter, n_pop, r_cross, r_mut):
    # Initialize population
    pop = [
        randint(0, 2, n_bits_cont * len(bounds_cont)).tolist() +
        randint(0, 2, n_bits_discret * bounds_discret).tolist()
        for _ in range(n_pop)
    ]

    # Decode first population and evaluate
    best, best_eval = 0, float('inf')
    for gen in range(n_iter):
        decoded = [
            decode_cont(bounds_cont, n_bits_cont, p[:n_bits_cont * len(bounds_cont)]) +
            decode_discrete(bounds_discret, n_bits_discret, p[n_bits_cont * len(bounds_cont):])
            for p in pop
        ]
        scores = [objective(d) for d in decoded]
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f">Gen {gen}, new best f({decoded[i]}) = {scores[i]}")

        # Select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]

        # Generate the next generation
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i + 1]
            for c in crossover(p1, p2, r_cross):# produces two childeren
                mutation(c, r_mut)# we need to check here if the child is valid
                children.append(c)

        pop = children  # Replace the population

    return best, best_eval



In [ ]:
# Parameters
n_bits_cont = 6  # Bits per continuous variable
n_bits_discret = 3  # Bits per discrete variable
bounds_cont = [(0.0, 10.0)] * 7  # Continuous variable bounds
bounds_discret = 8  # Number of discrete variables
n_iter = 10  # Number of generations
n_pop = 20  # Population size
r_cross = 0.9  # Crossover rate
r_mut = 0.1  # Mutation rate

# Run the Genetic Algorithm
best, best_eval = genetic_algorithm(
    objective, bounds_cont, n_bits_cont, bounds_discret, n_bits_discret,
    n_iter, n_pop, r_cross, r_mut
)
print(f"Best Solution: {best}, Best Objective: {best_eval}")

>Gen 0, new best f([5.0, 6.25, 7.5, 6.25, 8.75, 6.25, 5.0, 2, 4, 0, 0, 1, 6, 4, 7]) = 69.0
>Gen 0, new best f([2.5, 1.25, 8.75, 0.0, 6.25, 3.75, 5.0, 1, 0, 1, 0, 1, 0, 1, 7]) = 38.5
>Gen 0, new best f([0.0, 0.0, 0.0, 7.5, 1.25, 1.25, 6.25, 4, 2, 1, 1, 4, 1, 6, 0]) = 35.25
>Gen 1, new best f([5.0, 0.0, 0.0, 2.5, 7.5, 3.75, 3.75, 0, 2, 2, 0, 1, 0, 0, 3]) = 30.5
>Gen 3, new best f([1.25, 1.25, 2.5, 0.0, 1.25, 3.75, 1.25, 4, 2, 1, 6, 1, 0, 0, 2]) = 27.25
>Gen 5, new best f([0.0, 1.25, 2.5, 0.0, 1.25, 2.5, 1.25, 4, 1, 1, 0, 0, 4, 2, 6]) = 26.75
>Gen 5, new best f([1.25, 1.25, 0.0, 0.0, 1.25, 1.25, 1.25, 4, 2, 1, 6, 1, 1, 3, 2]) = 26.25
>Gen 7, new best f([1.25, 1.25, 1.25, 0.0, 1.25, 3.75, 1.25, 2, 2, 3, 4, 1, 0, 1, 2]) = 25.0
Best Solution: [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0], Best Objective: 25.0


In [ ]:
def decode_discrete(bounds_discrete, n_bits_discrete, bitstring):
    decoded_discrete = []
    start = 0  # Initialize the starting index

    for i in range(bounds_discrete):
        # Get the number of bits for the current step
        n_bits = n_bits_discrete[i]
        end = start + n_bits  # Calculate the end index for slicing

        # Extract the substring corresponding to the current discrete value
        substring = bitstring[start:end]
        chars = ''.join([str(s) for s in substring])  # Convert bits to string
        value = int(chars, 2)  # Convert binary string to integer
        decoded_discrete.append(value)

        # Update the start position for the next iteration
        start = end

    return decoded_discrete


In [ ]:
# Example inputs
bounds_discrete = 6
n_bits_discrete = [2, 3, 2, 3, 3,2]
bitstring = "110101100110101"

# Decode the bitstring
decoded_values = decode_discrete(bounds_discrete, n_bits_discrete, bitstring)
print(decoded_values)


[3, 2, 3, 1, 5, 1]
